In [1]:
from client import start_client
import numpy as np

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:32876' processes=5 threads=5, memory=33.69 GB>


In [2]:
from main_func import main
main(client,w)

workers: [<Future: pending, key: worker_f-d7ba01ccda04b8400d768882682ef2ae>, <Future: pending, key: worker_f-fda3a95e721632d38d3d56ac8fab65d8>, <Future: pending, key: worker_f-569fa2ada1222c2b51fbde28d56a2451>, <Future: pending, key: worker_f-430ad87c505492c967d22645a945cff0>]
['pending', 'pending', 'pending', 'pending', 'pending']
coo (array([   7.84936873,   24.73831384,  189.25725391, -112.09214551]), 1, 1)
coo (array([   7.84936873,   24.73831384,  189.25725391, -112.09214551]), 1, 1)
coo (array([ 101.38708505,   -6.23088026,  368.61450018, -147.29639718]), 1, 15)
coo (array([ 168.23812122,  -14.87346606,  582.77177643, -226.31667877]), 1, 1)
coo (array([ 210.44142599,   -8.44926101,  793.67738053, -323.6321318 ]), 1, 1)
coo (array([ 252.48827737,   -2.04887137, 1003.80112883, -420.58682319]), 1, 1)
coo (array([ 294.37983104,    4.32787879, 1213.14879681, -517.18341785]), 1, 1)
coo (array([ 376.42213403,  -12.98016612, 1433.26583843, -587.40589983]), 1, 1)
coo (array([ 446.21764012